In [ ]:
from Source_Model.MobileFormer import mobile_former
import torch.nn as nn
from Dataset.DataTorch import create_data
import torch
import torch.optim as optim
from tqdm import tqdm

In [ ]:
model=mobile_former.mobile_former_508m(img_size=24)

In [3]:
model.classifier.classifier=nn.Sequential(
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(1920,1024),
    nn.ReLU(),
    nn.Linear(1024,48)
)

In [ ]:
dataloader_dict=create_data()

In [ ]:
# Xác định hàm mất mát và thuật toán tối ưu
criterion = criterior = nn.L1Loss()
optimizer = optimizer = optim.Adam(model.parameters(), lr=0.00001)
def train_model(net, dataloader_dict, criterior, optimizer, num_epochs):
    for epoch in range(num_epochs):
        with open('output.txt', 'a') as f:
            print("Epoch {}/{}".format(epoch, num_epochs),file=f)
        
        for phase in ["Train", "Val"]:
            if phase == "Train":
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            for inputs, labels in tqdm(dataloader_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "Train"):
                    outputs = net(inputs.permute(0, 3, 1, 2))
                    loss = criterior(outputs, labels)
                    preds=outputs

                    
                    if phase == "Train":
                        loss.backward()
                        optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0) * inputs.size(1) * inputs.size(2)
                    epoch_corrects += torch.sum(torch.abs(preds - labels.data) < 1e-5)
                    
            
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_accuracy = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
                    
            with open('output.txt', 'a') as f:
                print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_accuracy), file=f)
        torch.save(model.state_dict(), 'mobileformer.pth')

In [ ]:
train_model(model,dataloader_dict,criterion,optimizer,100)